In [1]:
import os
os.chdir('..')

In [2]:
from tonal import pianoroll2number, joint_prob2pianoroll96
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pypianoroll import Multitrack, Track
import pypianoroll as pr
import pickle
from matplotlib import pyplot as plt
import os
import random
from model.CVAE import *
from decode import *

In [71]:
from metrics import CHE_and_CC, CTD, CTnCTR, PCS, MCTD

In [70]:
pwd

'/mnt/md1/user_victor/automatic_melody_harmonization'

In [10]:
device = 'cpu'

In [4]:
# Load data
device = 'cpu'
val_size = 500

print('loading data...')
melody_data = np.load('./melody_data.npy')
chord_groundtruth = np.load('./chord_groundtruth.npy')
chord_onehot = np.load('./onehot_96.npy')

# Reconstructed chords
chord_recon = np.load('./reconstructed_one_hot_chords.npy')

melody = np.load('./melody_baseline.npy')
lengths = np.load('./length.npy')

f = open('tempos', 'rb')
tempos = pickle.load(f)
f.close()
f = open('downbeats', 'rb')
downbeats = pickle.load(f)
f.close()

print('splitting testing set...')
melody_data = melody_data[:val_size]
chord_groundtruth = chord_groundtruth[:val_size]

val_chord = torch.from_numpy(chord_onehot[:val_size]).float()
val_melody = torch.from_numpy(melody[:val_size]).float()
val_length = torch.from_numpy(lengths[:val_size])

tempos = tempos[:val_size]
downbeats = downbeats[:val_size]

loading data...
splitting testing set...


In [60]:
print(val_melody.shape)
print(melody_data.shape)

torch.Size([500, 272, 576])
(500, 13056, 128)


In [5]:
# Load model
print('building model...')
model = CVAE(device = device).to(device)
model.load_state_dict(torch.load('output_models/model_cvae_reconstruction.pth'))

model.eval()
chord, length = val_chord.to(device), val_length.to(device)

building model...


In [60]:
# chord_groundtruth.shape
val_length[0].device

device(type='cpu')

In [22]:

latent_size = 16
pianoroll_frames = 12 * 24 * 2
index = 20

melody_truth = np.expand_dims(melody_data[index], axis=0)
chord_truth = np.expand_dims(chord_groundtruth[index], axis=0)
tempo = [tempos[index]]
downbeat = [downbeats[index]]

melody1 = val_melody[index].unsqueeze(dim=0)
length1 = torch.Tensor([length[index]]).long()


In [27]:
# Sampling
latent = torch.rand(1,272,latent_size)
z = torch.cat((latent,melody1), dim=-1)

_, chord_pred = model.decode(z,length1)

In [61]:
z.device

device(type='cpu')

In [9]:
melody_truth.shape

(1, 13056, 128)

In [62]:
chord_pred.shape

torch.Size([1, 272, 96])

In [86]:
downbeat.shape

(768,)

In [28]:
########## Random sampling ###########
# Proceed chord decode
print('proceed chord decode...')
length = length1
joint_prob = chord_pred.cpu().detach().numpy()

# Append argmax index to get pianoroll array
accompany_pianoroll = argmax2pianoroll(joint_prob)

# augment chord into frame base
BEAT_RESOLUTION = 24
BEAT_PER_CHORD = 2

accompany_pianoroll_frame, chord_groundtruth_frame = sequence2frame(accompany_pianoroll, chord_truth, BEAT_RESOLUTION=BEAT_RESOLUTION, BEAT_PER_CHORD=BEAT_PER_CHORD)

# length into frame base
length = length * BEAT_RESOLUTION * BEAT_PER_CHORD

# write pianoroll
result_dir = 'results/cvae_sample_result'
write_pianoroll(result_dir, melody_truth, accompany_pianoroll_frame,chord_groundtruth_frame, length, tempo,downbeat)


proceed chord decode...
(1, 272, 128)
augment chord into frame base...
accompany_pianoroll shape: (1, 13056, 128)
groundtruth_pianoroll shape: (1, 13056, 128)
write pianoroll...
Finished!


34.0

In [40]:
x = torch.empty(0)

In [67]:
print(chord_pred.shape)
print(length1.shape)

torch.Size([1, 272, 96])
torch.Size([1])


In [68]:
length1

tensor([33])

In [81]:
chord_pred_part = chord_pred[0][:val_length[0]].cpu().detach().numpy()
melody_part = melody[0][:val_length[0]]

In [82]:
print(chord_pred_part.shape) 
print(melody_part.shape)

(17, 96)
(17, 576)


In [79]:
che, cc = CHE_and_CC(chord_pred_part, chord_num=96)
ctd = CTD(chord_pred_part, chord_num=96)
ctnctr = CTnCTR(melody_part, chord_pred_part, chord_num=96)
pcs = PCS(melody_part, chord_pred_part, chord_num=96)
mctd = MCTD(melody_part, chord_pred_part, chord_num=96)

In [80]:
ctnctr

0.7381615598885793

In [83]:
weight_chord = np.load('./weight_chord.npy')

In [85]:
weight_chord

array([0.02845915, 0.03385304, 1.72714106, 1.00526172, 0.35642776,
       0.35397114, 0.19579946, 0.40494889, 0.52310953, 1.63496219,
       1.73572526, 1.43952153, 1.48734209, 0.92437248, 1.71022489,
       1.43833576, 0.43730018, 0.18795905, 1.74613962, 0.57533431,
       1.00352852, 1.65197693, 0.26484751, 0.68181945, 0.11128288,
       1.04747427, 1.74613962, 1.62734354, 1.17348092, 0.69099312,
       1.44309059, 1.05443214, 0.65669034, 0.24866699, 1.72203118,
       1.27176957, 1.43243611, 1.67254753, 0.45963138, 0.87833985,
       0.04619661, 0.13839578, 1.73056454, 1.28961567, 0.54841068,
       0.30484281, 0.32264221, 0.42923786, 1.06536889, 1.63496219,
       1.73572526, 0.94436972, 1.67254753, 1.44428422, 1.70023332,
       1.65667895, 0.05119893, 0.20268597, 1.71695144, 1.34525394,
       0.36446245, 1.5276812 , 0.34577022, 0.24873784, 0.06751758,
       1.23402093, 1.72885111, 1.51574619, 1.09407244, 0.23422396,
       1.50399622, 0.73060235, 0.8326846 , 0.0917042 , 1.73400